In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
train_df.head()

,Unnamed: 0,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label
0,0,"Mon, 6 Nov 2017 11:13:45 +0100",reply,ebay.in,0,0,multipart/alternative,35,120,0,0,49.0,80027,2
1,1,"Wed, 14 Feb 2018 11:00:16 -0000",edm,efinmail.com,0,0,multipart/alternative,1,7,0,0,107.0,2961,1
2,2,"Wed, 6 Jul 2016 19:53:37 +0000",usebackpack,com,0,0,text/html,4,17,0,0,35.0,25149,1
3,3,"Fri, 11 Oct 2019 11:25:40 +0200",granular,ai,0,0,multipart/mixed,0,0,0,0,15.0,635296,1
4,4,"Tue, 07 Nov 2017 11:07:18 +0000 (UTC)",github,com,1,0,multipart/alternative,2,11,0,0,49.0,2355,1


In [47]:
train_df.isna().sum()

org             0
ccs             0
bcced           0
images          0
urls            0
               ..
tld_back_so     0
tld_back_net    0
tld_back_au     0
tld_back_jp     0
tld_back_io     0
Length: 67, dtype: int64

In [4]:
import datetime
from datetime import datetime

def findd(df):
    index = df.find(',')
    if index == -1:
        index = 0
    else:
        index = index+2
    return index

def finddd(df):
    index1 = df.find('+')
    index2 = df.find('-')
    return max(index1, index2)

def fin(df):
    index = df.find(':')
    return index

def period(x):
  if x<11:
    return 'start'
  elif x<21:
    return 'middle'
  else:
    return 'end'

def determining_week(x):
  index = x.find('/')
  hour = int(x[:index])
  weekday = int(x[index+1:])
  if hour >= 9 and hour < 17 and weekday != 0 and weekday != 6:
    return 'weekday'
  elif hour >= 9 and hour < 17 and (weekday == 0 or weekday == 6):
    return 'weekend'
  else:
    return 'all week'

def determining_day(x):
  index = x.find('/')
  hour = int(x[:index])
  weekday = int(x[index+1:])
  if hour >= 9 and hour < 17 and weekday != 0 and weekday != 6:
    return 'working hours'
  elif hour >= 9 and hour < 17 and (weekday == 0 or weekday == 6):
    return 'weekend day'
  elif hour >= 17 and hour <= 23:
    return 'evening'
  elif hour <= 5:
    return 'night'
  else:
    return 'morning'


def feature_engineering(train_df):

  train_df['date'] = train_df['date'].apply(lambda x: x.replace("  "," ")) #some rows have extra space

  train_df['day'] = train_df['date'].apply(lambda x: x[findd(x):findd(x)+2].replace(" ", ""))
  train_df['day'] = train_df['day'].apply(lambda x: '0'+x if len(x)==1 else x) # to be used for datetime

  train_df['month'] = train_df['date'].apply(lambda x: x[findd(x)+2:findd(x)+6].replace(" ", ""))

  train_df['year'] = train_df['date'].apply(lambda x: x[findd(x)+6:findd(x)+11].replace(" ", ""))

  train_df['hour'] = train_df['date'].apply(lambda x: x[findd(x)+11:fin(x)].replace(" ", ""))

  train_df['minute'] = train_df['date'].apply(lambda x: x[fin(x)+1:fin(x)+3].replace(" ", ""))

  train_df['second'] = train_df['date'].apply(lambda x: x[fin(x)+4:fin(x)+6].replace(" ", ""))

  train_df['GMT'] = train_df['date'].apply(lambda x: x[finddd(x):].replace(" ", ""))

  return train_df

def feature_engineering_cleaning(train_df):
    # Feature Engineering after cleaning:
  # Example: maybe if promotion more images will be used than characters so proportion of char in body will be smaller

  train_df['month_number'] = train_df['month'].apply(lambda x: datetime.strptime(x,'%b').strftime('%m'))

  train_df['weekday'] = (train_df['day'] +'-'+ train_df['month'] +'-'+ train_df['year']).apply(lambda x: datetime.strptime(x, '%d-%b-%Y').strftime('%w'))
  # Weekday as a decimal number, where 0 is Sunday and 6 is Saturday.

  train_df['period_month'] = train_df['day'].astype(int).apply(lambda x: period(x))

  train_df['period_time'] = train_df['hour'].astype(int).apply(lambda x: 'AM' if x < 12 else 'PM')

  train_df['timezone'] = train_df['GMT'].apply(lambda x: x[:5]).apply(lambda x: '+0000' if len(x) != 5 else x).apply(lambda x: '+0000' if x == '-0000' else x)

  train_df['time'] = train_df['hour'] +'/'+ train_df['weekday']

  train_df['day_of_week'] = train_df['time'].apply(lambda x: determining_week(x))
  train_df['time_of_day'] = train_df['time'].apply(lambda x: determining_day(x))

  train_df.drop('time',axis= 1, inplace=True)

  train_df = data_cleaner(train_df)

  train_df['chars_in_subject'] = train_df['chars_in_subject'].astype(int)
  train_df['chars_in_body'] = train_df['chars_in_body'].astype(int)
  train_df['hour'] = train_df['hour'].astype(int)
  train_df['month_number'] = train_df['month_number'].astype(int)
  train_df['weekday'] = train_df['weekday'].astype(int)


  train_df['chars_in_subject_proportion'] = train_df['chars_in_subject'] / (train_df['chars_in_subject'] + train_df['chars_in_body'])
  train_df['chars_in_body_proportion'] = train_df['chars_in_body'] / (train_df['chars_in_subject'] + train_df['chars_in_body'])

  train_df['urls_images_proportion'] = (train_df['urls'] + train_df['images'])/ (train_df['chars_in_body'] + train_df['urls'] + train_df['images'])

  train_df['urls_proportion_body'] = train_df['urls']/ (train_df['chars_in_body'] + 1)

  train_df['images_proportion_body'] = train_df['images']/ (train_df['chars_in_body'] + 1)

  train_df['chars_in_subject_proportion'] = train_df['chars_in_subject_proportion'].astype(int)

  train_df['chars_in_body_proportion'] = train_df['chars_in_body_proportion'].astype(int)

  train_df['urls_images_proportion'] = train_df['urls_images_proportion'].astype(int)

  train_df['urls_proportion_body'] = train_df['urls_proportion_body'].astype(int)

  train_df['images_proportion_body'] = train_df['images_proportion_body'].astype(int)

  return train_df

In [5]:
def data_cleaner (df, chars_in_subject = train_df['chars_in_subject'].median() ):
    
    df['org'] = df['org'].fillna('org_undefinded')
    df['tld'] = df['tld'].fillna('tld_undefinded')
    df['mail_type'] = df['mail_type'].fillna('mail_type_undefinded').apply(lambda x:x.lower())
    df['chars_in_subject'] = df['chars_in_subject'].fillna(chars_in_subject)
    df.drop(['Unnamed: 0', 'date', 'GMT', 'day', 'month', 'year', 'minute', 'second'],axis= 1, inplace=True)
    return (df)

def invert_str(x):
    return(x[::-1])

In [6]:
# One hot encoding the mail_type
from sklearn.preprocessing import OneHotEncoder

def ohe_mail_type (df):
    unique_mail_types = df['mail_type'].apply(lambda x: x.split('/')).explode().unique()
    for mt in unique_mail_types:
        df[f'mail_type_{mt}'] = df['mail_type'].apply(lambda x: 1 if mt in x else 0)
    df.drop('mail_type',axis =1,inplace = True)
    return(df)

def ohe_others(df):

  unique_period_month = df['period_month'].unique()
  for mt in unique_period_month:
      df[f'period_month_{mt}'] = df['period_month'].apply(lambda x: 1 if mt in x else 0)
  df.drop('period_month',axis= 1, inplace=True)

  unique_period_time = df['period_time'].unique()
  for mt in unique_period_time:
      df[f'period_time_{mt}'] = df['period_time'].apply(lambda x: 1 if mt in x else 0)
  df.drop('period_time',axis= 1, inplace=True)

  unique_day_of_week = df['day_of_week'].unique()
  for mt in unique_day_of_week:
      df[f'day_of_week_{mt}'] = df['day_of_week'].apply(lambda x: 1 if mt in x else 0)
  df.drop('day_of_week',axis= 1, inplace=True)

  unique_time_of_day = df['time_of_day'].unique()
  for mt in unique_time_of_day:
      df[f'time_of_day_{mt}'] = df['time_of_day'].apply(lambda x: 1 if mt in x else 0)
  df.drop('time_of_day',axis= 1, inplace=True)

  return df

In [7]:
# Splitting tld
def tld_split(df):
    df['tld'].apply(lambda x:x.split('.')[0]).unique()
    df['tld_back'] = df['tld'].apply(lambda x: 'undefined_tld_back' if '.' not in x else invert_str(invert_str(x).split('.')[0].lower()))
    df['tld_front'] = df['tld'].apply(lambda x: 'undefined_tld_front' if '.' not in x else x.split('.')[0].lower())
    df.drop(['tld'], axis = 1, inplace = True)
    top_10_tld_back = pd.DataFrame(df[['tld_back']].groupby('tld_back').size()).sort_values(0,ascending=False).head(20).index.tolist()
    df['tld_back'] = df['tld_back'].apply(lambda x: x if x in top_10_tld_back else 'tld_back_Uncommon')
    for bk in df['tld_back'].unique().tolist():
        df[f'tld_back_{bk}'] = df['tld_back'].apply(lambda x: 1 if x == bk else 0 )
    df.drop(['tld_back'], axis = 1, inplace = True)
    return(df)

In [8]:
train_df = feature_engineering(train_df)

# Dropped 2 rows
train_df = train_df[train_df['year'] != '131']
train_df = train_df[train_df['month'] != '-MAR']

train_df = feature_engineering_cleaning(train_df)

In [9]:
train_df = ohe_mail_type(train_df)
train_df = ohe_others(train_df)
train_df = tld_split(train_df)

In [10]:
train_df.head(20)

,org,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,label,...,tld_back_co,tld_back_int,tld_back_edu,tld_back_uk,tld_back_sa,tld_back_so,tld_back_net,tld_back_au,tld_back_jp,tld_back_io
0,reply,0,0,35,120,0,0,49,80027,2,...,0,0,0,0,0,0,0,0,0,0
1,edm,0,0,1,7,0,0,107,2961,1,...,0,0,0,0,0,0,0,0,0,0
2,usebackpack,0,0,4,17,0,0,35,25149,1,...,0,0,0,0,0,0,0,0,0,0
3,granular,0,0,0,0,0,0,15,635296,1,...,0,0,0,0,0,0,0,0,0,0
4,github,1,0,2,11,0,0,49,2355,1,...,0,0,0,0,0,0,0,0,0,0
5,iiitd,5,0,2,14,1,0,78,8365,3,...,0,0,0,0,0,0,0,0,0,0
6,piazza,0,0,0,11,1,1,31,3357,1,...,0,0,0,0,0,0,0,0,0,0
7,in,0,0,0,4,1,1,218,15862,1,...,0,0,0,0,0,0,0,0,0,0
8,redwolf,0,0,14,39,0,0,49,8011,1,...,0,0,0,0,0,0,0,0,0,0
9,m,0,0,25,131,0,1,18,92248,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
from sklearn import preprocessing


In [12]:
le_org = preprocessing.LabelEncoder()
train_df['timezone'] = le_org.fit_transform(train_df['timezone'])

In [13]:
le_tldfront = preprocessing.LabelEncoder()
train_df['tld_front'] = le_tldfront.fit_transform(train_df['tld_front'])

In [16]:
for x in train_df.columns:
  if train_df[x].dtype != np.int64:
    print(x)

for x in train_df.columns:
  print(x + str(train_df[x].isna().sum()))

# Now we are sure that there are no NA values and all types are integers except org

org
label_0_probab
label_1_probab
label_2_probab
label_3_probab
label_4_probab
label_5_probab
label_6_probab
label_7_probab
org0
ccs0
bcced0
images0
urls0
salutations0
designation0
chars_in_subject0
chars_in_body0
label0
hour0
month_number0
weekday0
timezone0
chars_in_subject_proportion0
chars_in_body_proportion0
urls_images_proportion0
urls_proportion_body0
images_proportion_body0
mail_type_multipart0
mail_type_alternative0
mail_type_text0
mail_type_html0
mail_type_mixed0
mail_type_related0
mail_type_plain0
mail_type_mail_type_undefinded0
mail_type_html 0
mail_type_signed0
mail_type_report0
mail_type_calendar0
mail_type_idm0
period_month_start0
period_month_middle0
period_month_end0
period_time_AM0
period_time_PM0
day_of_week_weekday0
day_of_week_all week0
day_of_week_weekend0
time_of_day_working hours0
time_of_day_evening0
time_of_day_morning0
time_of_day_weekend day0
time_of_day_night0
tld_front0
tld_back_in0
tld_back_com0
tld_back_undefined_tld_back0
tld_back_org0
tld_back_me0
tld_

In [15]:
right_df = pd.read_csv('final_label_probab_df.csv')
train_df = train_df.merge(right_df, on='org', how='left', indicator=False)

In [78]:
train_df.head()

,org,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,label,...,tld_back_jp,tld_back_io,label_0_probab,label_1_probab,label_2_probab,label_3_probab,label_4_probab,label_5_probab,label_6_probab,label_7_probab
0,reply,0,0,35,120,0,0,49,80027,2,...,0,0,0.032828,0.497475,0.469697,0.000000,0.0,0.0,0.0,0.0
1,edm,0,0,1,7,0,0,107,2961,1,...,0,0,0.492857,0.450000,0.057143,0.000000,0.0,0.0,0.0,0.0
2,usebackpack,0,0,4,17,0,0,35,25149,1,...,0,0,0.448915,0.504142,0.046943,0.000000,0.0,0.0,0.0,0.0
3,granular,0,0,0,0,0,0,15,635296,1,...,0,0,0.026502,0.733216,0.001767,0.238516,0.0,0.0,0.0,0.0
4,github,1,0,2,11,0,0,49,2355,1,...,0,0,0.190280,0.182867,0.000000,0.626853,0.0,0.0,0.0,0.0


In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_df.drop(['label', 'org'],axis = 1), train_df['label'], test_size=0.2, random_state=42)

In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(n_estimators=500,
                             max_depth=700, 
                             random_state=0, 
                             criterion='gini')

In [92]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler , LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
from scipy.stats import mode



clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(f'Acuracy of the Random Forest: {accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))

KeyboardInterrupt: ignored

In [83]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
pred = neigh.predict(X_test)
print(f'Acuracy of the KNN3: {accuracy_score(y_test, pred)}')

Acuracy of the KNN3: 0.3993140006236358


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X_train, y_train)
pred = neigh.predict(X_test)
print(f'Acuracy of the KNN4:{accuracy_score(y_test, pred)}')

Acuracy of the KNN4:0.4479383196643864


In [85]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
pred = neigh.predict(X_test)
print(f'Acuracy of the KNN5:{accuracy_score(y_test, pred)}')

Acuracy of the KNN5:0.432304334268787


In [88]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'n_neighbors':list(range(14,20))}
neigh = KNeighborsClassifier()
clf = GridSearchCV(neigh, parameters)
clf.fit(X_train, y_train)
print(clf.best_params_)
pred = clf.predict(X_test)
print(f'Acuracy of the KNN{clf.best_params_}:{accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))
#sorted(clf.cv_results_.keys())


{'n_neighbors': 18}
Acuracy of the KNN{'n_neighbors': 18}:0.4671655753040225
              precision    recall  f1-score   support

           0       0.45      0.42      0.44      3638
           1       0.48      0.65      0.55      7399
           2       0.34      0.11      0.17      1723
           3       0.51      0.35      0.42      2115
           4       0.12      0.01      0.02        72
           5       0.25      0.03      0.06        30
           6       0.00      0.00      0.00        27
           7       0.37      0.22      0.28      1031

    accuracy                           0.47     16035
   macro avg       0.31      0.23      0.24     16035
weighted avg       0.45      0.47      0.44     16035



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [95]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=18)
neigh.fit(X_train, y_train)
pred = neigh.predict(X_test)
print(f'Acuracy of the KNN18:{accuracy_score(y_test, pred)}')
confusion_matrix(y_test, pred)
classification_report(y_test, pred)
#precision_score(y_test, pred)
#recall_score(y_test, pred)

Acuracy of the KNN18:0.4671655753040225


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.45      0.42      0.44      3638\n           1       0.48      0.65      0.55      7399\n           2       0.34      0.11      0.17      1723\n           3       0.51      0.35      0.42      2115\n           4       0.12      0.01      0.02        72\n           5       0.25      0.03      0.06        30\n           6       0.00      0.00      0.00        27\n           7       0.37      0.22      0.28      1031\n\n    accuracy                           0.47     16035\n   macro avg       0.31      0.23      0.24     16035\nweighted avg       0.45      0.47      0.44     16035\n'

In [84]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(f'Acuracy of the dtree:{accuracy_score(y_test, pred)}')

Acuracy of the dtree:0.40929217337075147


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=1000, learning_rate=2.0, random_state=0)
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(f'Acuracy of the adaboost:{accuracy_score(y_test, pred)}')

Acuracy of the adaboost:0.11124968820154651


In [ ]:
# BEFORE running the below code, you need to retrain your model by doing this:
X = train_df.drop(['label', 'org'],axis = 1)
y = train_df['label']

# If you used pca you have to re-do it here with X and y (same for scaling) before .fit

# Better to redefine before .fit the clf, for example clf = AdaBoostClassifier(n_estimators=1000, learning_rate=2.0, random_state=0)

clf = clf.fit(X, y)

In [34]:
test_df = pd.read_csv('test.csv') # control shift L

test_df = feature_engineering(test_df)

# ONLY FOR TRAINING SET
# A date was 13 instead of 2013
test_df['year'] = test_df['year'].apply(lambda x: x[1:] if len(x)==5 else x)
test_df['month'] = test_df['month'].apply(lambda x: 'Mar' if len(x)!=3 else x)

test_df = feature_engineering_cleaning(test_df)

In [35]:
test_df.head()

,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,...,period_month,period_time,timezone,day_of_week,time_of_day,chars_in_subject_proportion,chars_in_body_proportion,urls_images_proportion,urls_proportion_body,images_proportion_body
0,iiitd,ac.in,0,0,multipart/alternative,0,12,1,0,20,...,start,PM,+0530,weekday,working hours,0,0,0,0,0
1,notifications,freelancer.com,0,0,multipart/alternative,7,49,1,0,49,...,start,PM,-0500,all week,evening,0,0,0,0,0
2,iiitd,ac.in,1,0,multipart/alternative,0,0,1,0,24,...,end,PM,+0530,weekday,working hours,0,0,0,0,0
3,udacity,com,0,0,multipart/alternative,13,50,1,0,70,...,start,PM,+0000,weekday,working hours,0,0,0,0,0
4,google,com,0,0,text/html,18,76,1,0,38,...,end,AM,-0700,all week,night,0,0,0,0,0


In [37]:
test_df = ohe_mail_type(test_df)
test_df = ohe_others(test_df)
test_df = tld_split(test_df)

In [40]:
le_org = preprocessing.LabelEncoder()
test_df['timezone'] = le_org.fit_transform(test_df['timezone'])

le_tldfront = preprocessing.LabelEncoder()
test_df['tld_front'] = le_tldfront.fit_transform(test_df['tld_front'])

In [41]:
for x in test_df.columns:
  if test_df[x].dtype != np.int64:
    print(x)

for x in test_df.columns:
  print(x + str(test_df[x].isna().sum()))

# Now we are sure that there are no NA values and all types are integers except org

org
org0
ccs0
bcced0
images0
urls0
salutations0
designation0
chars_in_subject0
chars_in_body0
hour0
month_number0
weekday0
timezone0
chars_in_subject_proportion0
chars_in_body_proportion0
urls_images_proportion0
urls_proportion_body0
images_proportion_body0
mail_type_multipart0
mail_type_alternative0
mail_type_text0
mail_type_html0
mail_type_mixed0
mail_type_plain0
mail_type_mail_type_undefinded0
mail_type_related0
mail_type_html 0
mail_type_report0
mail_type_signed0
mail_type_idm0
period_month_start0
period_month_end0
period_month_middle0
period_time_PM0
period_time_AM0
day_of_week_weekday0
day_of_week_all week0
day_of_week_weekend0
time_of_day_working hours0
time_of_day_evening0
time_of_day_night0
time_of_day_weekend day0
time_of_day_morning0
tld_front0
tld_back_in0
tld_back_com0
tld_back_undefined_tld_back0
tld_back_fr0
tld_back_org0
tld_back_de0
tld_back_il0
tld_back_me0
tld_back_info0
tld_back_so0
tld_back_edu0
tld_back_uk0
tld_back_mil0
tld_back_tld_back_Uncommon0
tld_back_name0


In [42]:
right_df = pd.read_csv('final_label_probab_df.csv')
test_df = test_df.merge(right_df, on='org', how='left', indicator=False)